In [28]:
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
import requests
import pandas as pd
import seaborn as sns

In [ ]:

url = 'https://www.automobile.tn/fr/occasion'
nbre_page=175
n=1 #page 1
total_df = pd.DataFrame()

In [30]:
roads=[]
years=[]
boites=[]
transmissions=[]
horsepowers=[]
fuels=[]
nbre_portes=[]
cylindres=[]
items = []
marques = []
modeles = []
prices = []
links = []
features_tot=[]
features = []


In [ ]:
import re


for n in range(nbre_page):

    url = 'https://www.automobile.tn/fr/occasion/'+str(n)
    code_page=requests.get(url)

    soup= BeautifulSoup(code_page.text, 'html.parser')

    container=soup.findAll(class_='occasion-item-v2')

    for item in container:
        road=item.find('li',class_='road').text.strip()
        roads.append(road)

        year=item.find('li',class_='year').text.strip()
        years.append(year)

        boite=item.find('li',class_='boite').text.strip()
        boites.append(boite)

        transmission=item.find('li',class_='transmission').text.strip()
        transmissions.append(transmission)

        horsepower=item.find('li',class_='horsepower').text.strip()
        horsepowers.append(horsepower)

        fuel=item.find('li',class_='fuel').text.strip()
        fuels.append(fuel)



        #extraire la marque et le modele(sous h2)
        noms = item.findAll('h2')
        for nom in noms:
            nom_car = nom.text.strip()
            if nom_car:
                items.append(nom_car)
                marques.append(nom_car.split()[0])
                parts = nom_car.split()
                modeles.append(' '.join(parts[1:]))
        prices_dirty=item.findAll('div',class_='price')
        for p in prices_dirty:
            if p:
                price_final = p.text.replace('DT', '').replace('\xa0', '').strip()
                prices.append(price_final)

        link_tag = item.find('a', class_='occasion-link-overlay', href=True)
        if link_tag:
            link = link_tag['href']
            correct_link='https://www.automobile.tn/'+link
            links.append(correct_link)

    for link in links:
        page=requests.get(link)
        soup= BeautifulSoup(page.text, 'html.parser')
        container_a=soup.findAll('span',class_='spec-name') 

        for item in container_a:
            if item.text.strip() == 'Nombre de portes':
                if item.findNext('span').text :
                    nbre_portes.append(item.findNext('span').text.strip())
                else:
                    nbre_portes.append('4')
  
            if item.text.strip() == 'Cylindrée':
                cylindree_number = re.search(r'\d+', item.findNext('span').text).group()
                if cylindree_number:
                    cylindres.append(cylindree_number)
                            
                else:
                    cylindres.append('1')
        
        
    if len(prices) == len(marques) == len(modeles) == len(roads) == len(years) == len(boites) == len(transmissions) == len(horsepowers) == len(fuels) == len(nbre_portes) == len(cylindres) == len(prices):
        data = {'Marque': marques, 'Modèle': modeles, 'road': roads, 'year': years, 'boite': boites, 'transmission': transmissions, 'horsepower': horsepowers, 'fuel': fuels, 'Nombre de portes': nbre_portes, 'Cylindrée': cylindres,'Prix': prices}
        df = pd.DataFrame(data)
        print(df)
    
    total_df = pd.concat([total_df, df], ignore_index=True)

    print('-------------------------------------------------------------------------------------------------')
print(nbre_portes)
print(total_df)


In [52]:

#la dataframe totale contenant les données scrapées de toutes les pages:
total_df.to_csv('../data/cars.csv', index=False)
print(total_df)


           Marque                  Modèle        road  year        boite  \
0   Mercedes-Benz                 GLA AMG  150 000 km  2017  Automatique   
1         Peugeot                     208  110 000 km  2015     Manuelle   
2             BMW  Série 3 coupé Kit M///  152 000 km  2012     Manuelle   
3   Mercedes-Benz            Classe C AMG  200 000 km  2011  Automatique   
4          Toyota              C-HR Sport  162 000 km  2020  Automatique   
5            Seat                   Ibiza   98 000 km  2019     Manuelle   
6          Toyota                    C-HR   80 000 km  2020  Automatique   
7   Mercedes-Benz            Classe E AMG   98 000 km  2021  Automatique   
8        Mahindra             KUV 100 K6+   47 000 km  2021     Manuelle   
9      Volkswagen                  Tiguan  123 000 km  2021     Manuelle   
10           Fiat           500 Dolcevita   78 000 km  2021     Manuelle   
11           Fiat           Tipo 5 portes   68 000 km  2021     Manuelle   
12  Mercedes